# Configuração, Inicialização e Carregamento de Dados

In [1]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução

In [2]:
import pyspark # Importa a biblioteca principal do PySpark
from pyspark.sql import SparkSession # Importa a classe SparkSession, o ponto de entrada para o Spark
from pyspark.sql.functions import * # Importa todas as funções de agregação, formatação e manipulação de colunas (countDistinct, avg, var_samp, format_number, year, first, last, etc.)

In [3]:
from google.colab import drive # Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive no ambiente

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [5]:
clientes = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True) # Lê o DataFrame de clientes
itens_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de itens_pedido
pagamentos_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pagamentos_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame de pagamentos_pedido
pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True) # Lê o DataFrame de pedidos

In [6]:
clientes.show(2) # Exibe as 2 primeiras linhas do DataFrame clientes
pagamentos_pedido.show(2) # Exibe as 2 primeiras linhas do DataFrame pagamentos_pedido
pedidos.show(2) # Exibe as 2 primeiras linhas do DataFrame pedidos
itens_pedido.show(2) # Exibe as 2 primeiras linhas do DataFrame itens_pedido

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
+--------------------+--------------------+-----------+--------------------+--------------+
only showing top 2 rows

+--------------------+-------------------+--------------+------------------+---------------+
|           id_pedido|sequencia_pagamento|tipo_pagamento|parcelas_pagamento|valor_pagamento|
+--------------------+-------------------+--------------+------------------+---------------+
|b81ef226f3fe1789b...|                  1|   credit_card|                 8|          99.33|
|a9810da82917af2d9...|                  1|   credit

# Contagem e Estatísticas Financeiras

In [7]:
clientes.agg(countDistinct(col('estado_cliente')).alias('qtd_estados_distintos')).show() # Usa a função agg e countDistinct para contar o número de valores únicos na coluna estado_cliente e renomeia o resultado.

+---------------------+
|qtd_estados_distintos|
+---------------------+
|                   27|
+---------------------+



In [8]:
# Agrupa por tipo_pagamento e usa agg para calcular o valor médio (avg) e a variância amostral (var_samp) do valor_pagamento
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    avg('valor_pagamento').alias('pagamento médio'),
    var_samp('valor_pagamento').alias('pagamento variância')
).show()

+--------------+------------------+-------------------+
|tipo_pagamento|   pagamento médio|pagamento variância|
+--------------+------------------+-------------------+
|        boleto| 145.0344354023452| 45616.869821012544|
|   not_defined|               0.0|                0.0|
|   credit_card|163.31902063935797|  49336.98820414979|
|       voucher| 65.70335411255421| 13344.682202516573|
|    debit_card|142.57017004578165| 60414.395995128165|
+--------------+------------------+-------------------+



In [9]:
# Repete a agregação anterior, mas envolve os resultados com format_number(..., 2) para exibir a média e a variância com apenas duas casas decimais, melhorando a leitura.
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    format_number(avg('valor_pagamento'),2).alias('pagamento médio'),
    format_number(var_samp('valor_pagamento'),2).alias('pagamento variância')
).show()

+--------------+---------------+-------------------+
|tipo_pagamento|pagamento médio|pagamento variância|
+--------------+---------------+-------------------+
|        boleto|         145.03|          45,616.87|
|   not_defined|           0.00|               0.00|
|   credit_card|         163.32|          49,336.99|
|       voucher|          65.70|          13,344.68|
|    debit_card|         142.57|          60,414.40|
+--------------+---------------+-------------------+



# Análise Temporal e Sequencial

In [10]:
pedidos.printSchema() # Imprime o esquema (schema) do DataFrame pedidos para verificar os tipos de dados das colunas de data

root
 |-- id_pedido: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- data_compra_pedido: timestamp (nullable = true)
 |-- data_aprovacao_pedido: timestamp (nullable = true)
 |-- data_envio_transportadora: timestamp (nullable = true)
 |-- data_entrega_cliente: timestamp (nullable = true)
 |-- data_estimada_entrega: timestamp (nullable = true)



In [11]:
pedidos_ordenados = pedidos.orderBy('data_compra_pedido') # Cria um novo DataFrame ordenado pela coluna de data de compra

In [12]:
# Agrupa por status_pedido no DataFrame ordenado. Usa first e last para encontrar a data de compra mais antiga (primeiro) e mais recente (ultimo) para cada status de pedido
datas_compras_df = pedidos_ordenados.groupBy('status_pedido').agg(
    first('data_compra_pedido').alias('primeiro'),
    last('data_compra_pedido').alias('ultimo'))

datas_compras_df.show() # Exibe o primeiro e o último registro de compra por status

+-------------+-------------------+-------------------+
|status_pedido|           primeiro|             ultimo|
+-------------+-------------------+-------------------+
|      shipped|2016-09-04 21:15:19|2018-09-03 09:06:57|
|     canceled|2016-09-05 00:15:34|2018-10-17 17:30:18|
|     approved|2017-02-06 20:18:17|2017-04-25 01:25:34|
|     invoiced|2016-10-04 13:02:10|2018-08-14 18:45:08|
|      created|2017-11-06 13:12:34|2018-02-09 17:21:04|
|    delivered|2016-09-15 12:16:38|2018-08-29 15:00:37|
|  unavailable|2016-10-05 14:16:28|2018-08-21 12:21:00|
|   processing|2016-10-05 22:44:13|2018-07-23 18:03:03|
+-------------+-------------------+-------------------+



In [13]:
pedidos.withColumn("compra_anual", year('data_compra_pedido')).groupBy('compra_anual').count().orderBy(asc('compra_anual')).show() # Extrai o ano da data de compra, agrupa pelo ano e conta o número de pedidos, ordenando o resultado do ano mais antigo para o mais recente (asc)

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2016|  329|
|        2017|45101|
|        2018|54011|
+------------+-----+



In [14]:
pedidos.withColumn('compra_anual', year('data_compra_pedido')).groupBy('compra_anual').count().orderBy(desc('compra_anual')).show() # Repete a contagem anual, mas ordena o resultado do ano mais recente para o mais antigo (desc)

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2018|54011|
|        2017|45101|
|        2016|  329|
+------------+-----+



# Agregação de Valores em Pedidos

In [15]:
itens_pedido.groupBy('id_pedido').agg(
    sum(col('preco') + col('valor_frete')), # Agrupa os itens pelo id_pedido e calcula a soma do preco e do valor_frete (o valor total do pedido)
   sum(col('preco') + col('valor_frete')).alias('valor_total_pedido'), # Repete a soma, mas renomeia a coluna resultante para valor_total_pedido
   format_number(sum(col('preco') + col('valor_frete')),2).alias('valor_total_pedido') # Repete a soma e renomeia, mas formata o valor total com duas casas decimais
).show()

+--------------------+--------------------------+------------------+------------------+
|           id_pedido|sum((preco + valor_frete))|valor_total_pedido|valor_total_pedido|
+--------------------+--------------------------+------------------+------------------+
|014405982914c2cde...|         78.42999999999999| 78.42999999999999|             78.43|
|019886de8f385a39b...|                     188.4|             188.4|            188.40|
|01a6ad782455876aa...|                     50.09|             50.09|             50.09|
|01d907b3e209269e1...|        169.76000000000002|169.76000000000002|            169.76|
|028dc52e12ddda803...|                     61.72|             61.72|             61.72|
|036dd381dfb3ec75e...|                     69.14|             69.14|             69.14|
|03ebfa9712b7dbc70...|                     55.78|             55.78|             55.78|
|0420da8d50a378401...|                     74.63|             74.63|             74.63|
|05afef1c185862cab...|          

# Encerramento da Sessão

In [16]:
spark.stop() # Encerra a SparkSession e libera os recursos alocados